In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import math
import matplotlib.dates as mdate
import numpy as np
import datetime as dt
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


# final df

In [97]:
df = pd.read_csv('/Users/isdc/Library/CloudStorage/OneDrive-HertieSchool/2024-1/Machine Learning/ML project/machine_learning/all_features_merged.csv')
df.set_index(['Date', 'Country'], inplace=True)

print(df)

                             Solar  Wind Onshore  Wind Offshore  \
Date                Country                                       
2014-12-31 23:00    GB         0.0       4546.00         3165.0   
2015-01-01 00:00    AT         0.0        117.25            NaN   
                    BE         0.0        246.00          420.0   
                    CH         0.0          4.00            NaN   
                    CZ         0.0           NaN            NaN   
...                            ...           ...            ...   
2023-12-31 00:00:00 SE_2       NaN           NaN            NaN   
                    SE_3       NaN           NaN            NaN   
                    SE_4       NaN           NaN            NaN   
                    SI         NaN           NaN            NaN   
                    SK         NaN           NaN            NaN   

                             water_reservoirs_and_hydro_storage  energy_price  \
Date                Country                    

Until 2018, the energy price market had a joint trading zone between Austria, Germany and Luxembourg. After 2018, a separate market appeared in Austria. So we need to merge the columns for them.

In [98]:
df.describe()

,Solar,Wind Onshore,Wind Offshore,water_reservoirs_and_hydro_storage,energy_price,biomass,gas,nuclear
count,1.694963e+06,1.895443e+06,591053.000000,1.196200e+04,3.371340e+06,2.022328e+06,2.227018e+06,820910.000000
mean,3.768433e+02,8.095530e+02,476.024976,3.589091e+06,7.913296e+01,4.019074e+02,1.098890e+03,7085.624705
std,1.153057e+03,1.751954e+03,1042.416130,5.358699e+06,1.196151e+02,1.051868e+03,1.994761e+03,11559.506233
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,-5.000000e+02,0.000000e+00,0.000000e+00,0.000000
25%,0.000000e+00,3.200000e+01,0.000000,3.045920e+05,3.237000e+01,2.775000e+01,2.250000e+01,1595.312500
50%,0.000000e+00,1.710000e+02,91.000000,1.598700e+06,4.794000e+01,7.500000e+01,2.810000e+02,3001.000000
75%,1.932500e+02,7.050000e+02,489.000000,4.036250e+06,7.990000e+01,2.660000e+02,1.204000e+03,6420.000000
max,1.702500e+04,2.015600e+04,9813.000000,3.338800e+07,6.101780e+03,5.368250e+03,2.045400e+04,61490.000000


# Handling missing values

In [99]:
#df.dropna(inplace=True)
#print(df)

We cant drop missing values because we end up with nothing.

In [100]:
# Overall missing values
total_missing = df.isnull().sum().sum()
print(f"Total missing values in the dataset: {total_missing}")

Total missing values in the dataset: 31773487


In [101]:
# Missing values per column
missing_per_column = df.isnull().sum()
print(missing_per_column)

Solar                                 3856100
Wind Onshore                          3655620
Wind Offshore                         4960010
water_reservoirs_and_hydro_storage    5539101
energy_price                          2179723
biomass                               3528735
gas                                   3324045
nuclear                               4730153
dtype: int64


In [102]:
# Drop rows where the 'energy_price' column has NaN values
df_cleaned = df.dropna(subset=['energy_price'])

print(df_cleaned)


                             Solar  Wind Onshore  Wind Offshore  \
Date                Country                                       
2015-01-01 00:00:00 CH         NaN           NaN            NaN   
                    CZ         NaN           NaN            NaN   
                    DK_1       NaN           NaN            NaN   
                    DK_2       NaN           NaN            NaN   
                    EE         NaN           NaN            NaN   
...                            ...           ...            ...   
2023-12-31 00:00:00 SE_2       NaN           NaN            NaN   
                    SE_3       NaN           NaN            NaN   
                    SE_4       NaN           NaN            NaN   
                    SI         NaN           NaN            NaN   
                    SK         NaN           NaN            NaN   

                             water_reservoirs_and_hydro_storage  energy_price  \
Date                Country                    

## missing values

In [103]:
# we need to reset the index temporarily

# Reset index to use 'Date' as a column for resampling
#df.reset_index(inplace=True)

# Calculate monthly means, grouping by Country and resampling by month start
# monthly_means = df.groupby('Country').resample('MS', on='Date').mean()


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

## cross-validation using sklearn
###### (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html)


In [ ]:
# Sample DataFrame

# Sort by date
df.sort_index(level='Date', inplace=True)

X = df_cleaned.drop('energy_price', axis=1)
y = df_cleaned['energy_price']

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=20)

# Apply TimeSeriesSplit to df
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    print(f"Fold {i}:")
    print(f"  Train Indices: {train_index}")
    print(f"  Test Indices: {test_index}")


In [ ]:
# Calculate the covariance matrix for the training data
cov_matrix_train = X_train.cov()

# Display the covariance matrix for the training data
print("Covariance Matrix for Training Data:")
print(cov_matrix_train)

# Calculate the covariance matrix for the testing data
cov_matrix_test = X_test.cov()

# Display the covariance matrix for the testing data
print("Covariance Matrix for Testing Data:")
print(cov_matrix_test)


In [ ]:
# Assuming cov_matrix_train and cov_matrix_test are already computed
# Plotting the covariance matrix for training data
plt.figure(figsize=(10, 8))
sns.heatmap(cov_matrix_train, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Covariance Matrix - Training Data')
plt.show()

# Plotting the covariance matrix for testing data
plt.figure(figsize=(10, 8))
sns.heatmap(cov_matrix_test, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Covariance Matrix - Testing Data')
plt.show()
